# Let's get started with Method 1...

If you are running this notebook on Google Colab, run the following cell:

In [ ]:
!pip install py2neo
!pip install wikipedia
!pip install spacy==3.0.3

In [4]:
!pip install scikit-learn

You should consider upgrading via the '/Users/davidoluyalegbenga/.pyenv/versions/3.9.6/bin/python3.9 -m pip install --upgrade pip' command.


In [5]:
import json
import re
import urllib
from pprint import pprint
import time
from tqdm import tqdm

from py2neo import Node, Graph, Relationship, NodeMatcher
from py2neo.bulk import merge_nodes

import numpy as np
import pandas as pd
import wikipedia
from sklearn.metrics.pairwise import cosine_similarity

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.matcher import Matcher
from spacy.tokens import Doc, Span, Token

print(spacy.__version__)

3.0.3


In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
SUBJECTS = ["nsubj", "nsubjpass", "csubj", "csubjpass", "agent", "expl"]
VERBS = ['ROOT', 'advcl']
OBJECTS = ["dobj", "dative", "attr", "oprd", 'pobj']
ENTITY_LABELS = ['PERSON', 'NORP', 'GPE', 'ORG', 'FAC', 'LOC', 'PRODUCT', 'EVENT', 'WORK_OF_ART']

api_key = open('../.api_key').read()

non_nc = spacy.load('en_core_web_md')

nlp = spacy.load('en_core_web_md')
nlp.add_pipe('merge_noun_chunks')

print(non_nc.pipe_names)
print(nlp.pipe_names)

# Query Google Knowledge Graph

 

In [15]:
def query_google(query, api_key, limit=10, indent=True, return_lists=True):
    
    text_ls = []
    node_label_ls = []
    url_ls = []
    
    params = {
        'query': query,
        'limit': limit,
        'indent': indent,
        'key': api_key,
    }   
    
    service_url = 'https://kgsearch.googleapis.com/v1/entities:search'
    url = service_url + '?' + urllib.parse.urlencode(params)
    response = json.loads(urllib.request.urlopen(url).read())
    
    if return_lists:
        for element in response['itemListElement']:

            try:
                node_label_ls.append(element['result']['@type'])
            except:
                node_label_ls.append('')

            try:
                text_ls.append(element['result']['detailedDescription']['articleBody'])
            except:
                text_ls.append('')
                
            try:
                url_ls.append(element['result']['detailedDescription']['url'])
            except:
                url_ls.append('')
                
        return text_ls, node_label_ls, url_ls
    
    else:
        return response

# Helper functions for text cleaning
 

In [16]:
def remove_special_characters(text):
    
    regex = re.compile(r'[\n\r\t]')
    clean_text = regex.sub(" ", text)
    
    return clean_text


def remove_stop_words_and_punct(text, print_text=False):
    
    result_ls = []
    rsw_doc = non_nc(text)
    
    for token in rsw_doc:
        if print_text:
            print(token, token.is_stop)
            print('--------------')
        if not token.is_stop and not token.is_punct:
            result_ls.append(str(token))
    
    result_str = ' '.join(result_ls)

    return result_str


def create_svo_lists(doc, print_lists):
    
    subject_ls = []
    verb_ls = []
    object_ls = []

    for token in doc:
        if token.dep_ in SUBJECTS:
            subject_ls.append((token.lower_, token.idx))
        elif token.dep_ in VERBS:
            verb_ls.append((token.lemma_, token.idx))
        elif token.dep_ in OBJECTS:
            object_ls.append((token.lower_, token.idx))

    if print_lists:
        print('SUBJECTS: ', subject_ls)
        print('VERBS: ', verb_ls)
        print('OBJECTS: ', object_ls)
    
    return subject_ls, verb_ls, object_ls


def remove_duplicates(tup, tup_posn):
    
    check_val = set()
    result = []
    
    for i in tup:
        if i[tup_posn] not in check_val:
            result.append(i)
            check_val.add(i[tup_posn])
            
    return result


def remove_dates(tup_ls):
    
    clean_tup_ls = []
    for entry in tup_ls:
        if not entry[2].isdigit():
            clean_tup_ls.append(entry)
    return clean_tup_ls


def create_svo_triples(text, print_lists=False):
    
    clean_text = remove_special_characters(text)
    doc = nlp(clean_text)
    subject_ls, verb_ls, object_ls = create_svo_lists(doc, print_lists=print_lists)
    
    graph_tup_ls = []
    dedup_tup_ls = []
    clean_tup_ls = []
    
    for subj in subject_ls: 
        for obj in object_ls:
            
            dist_ls = []
            
            for v in verb_ls:
                
                # Assemble a list of distances between each object and each verb
                dist_ls.append(abs(obj[1] - v[1]))
                
            # Get the index of the verb with the smallest distance to the object 
            # and return that verb
            index_min = min(range(len(dist_ls)), key=dist_ls.__getitem__)
            
            # Remve stop words from subjects and object.  Note that we do this a bit
            # later down in the process to allow for proper sentence recognition.

            no_sw_subj = remove_stop_words_and_punct(subj[0])
            no_sw_obj = remove_stop_words_and_punct(obj[0])
            
            # Add entries to the graph iff neither subject nor object is blank
            if no_sw_subj and no_sw_obj:
                tup = (no_sw_subj, verb_ls[index_min][0], no_sw_obj)
                graph_tup_ls.append(tup)
        
        #clean_tup_ls = remove_dates(graph_tup_ls)
    
    dedup_tup_ls = remove_duplicates(graph_tup_ls, 2)
    clean_tup_ls = remove_dates(dedup_tup_ls)
    
    return clean_tup_ls

# helper functions


In [43]:
def get_obj_properties(tup_ls):
    
    init_obj_tup_ls = []
    
    for tup in tup_ls:

        try:
            text, node_label_ls, url = query_google(tup[2], api_key, limit=1)
            new_tup = (tup[0], tup[1], tup[2], text[0], node_label_ls[0], url[0])
        except:
            new_tup = (tup[0], tup[1], tup[2], [], [], [])
        
        init_obj_tup_ls.append(new_tup)
        
    return init_obj_tup_ls


def add_layer(tup_ls):

    svo_tup_ls = []
    
    for tup in tup_ls:
        
        if tup[3]:
            svo_tup = create_svo_triples(tup[3])
            svo_tup_ls.extend(svo_tup)
        else:
            continue
    
    return get_obj_properties(svo_tup_ls)
        

def subj_equals_obj(tup_ls):
    
    new_tup_ls = []
    
    for tup in tup_ls:
        if tup[0] != tup[2]:
            new_tup_ls.append((tup[0], tup[1], tup[2], tup[3], tup[4], tup[5]))
            
    return new_tup_ls


def check_for_string_labels(tup_ls):
    # This is for an edge case where the object does not get fully populated
    # resulting in the node labels being assigned to string instead of list.
    # This may not be strictly necessary and the lines using it are commnted out
    # below.  Run this function if you come across this case.
    
    clean_tup_ls = []
    
    for el in tup_ls:
        if isinstance(el[2], list):
            clean_tup_ls.append(el)
            
    return clean_tup_ls


def create_word_vectors(tup_ls):

    new_tup_ls = []
    
    for tup in tup_ls:
        if tup[3]:
            doc = nlp(tup[3])
            new_tup = (tup[0], tup[1], tup[2], tup[3], tup[4], tup[5], doc.vector)
        else:
            new_tup = (tup[0], tup[1], tup[2], tup[3], tup[4], tup[5], np.random.uniform(low=-1.0, high=1.0, size=(300,)))
        new_tup_ls.append(new_tup)
        
    return new_tup_ls

# Let's now run this step by step...

In [48]:
def dedup(tup_ls):
    
    visited = set()
    output_ls = []
    
    for tup in tup_ls:
        if not tup[0] in visited:
            visited.add(tup[0])
            output_ls.append((tup[0], tup[1], tup[2], tup[3], tup[4]))
            
    return output_ls


def convert_vec_to_ls(tup_ls):
    
    vec_to_ls_tup = []
    
    for el in tup_ls:
        vec_ls = [float(v) for v in el[4]]
        tup = (el[0], el[1], el[2], el[3], vec_ls)
        vec_to_ls_tup.append(tup)
        
    return vec_to_ls_tup


def add_nodes(tup_ls):   

    keys = ['name', 'description', 'node_labels', 'url', 'word_vec']
    merge_nodes(graph.auto(), tup_ls, ('Node', 'name'), keys=keys)
    print('Number of nodes in graph: ', graph.nodes.match('Node').count())
    
    return

In [50]:
def add_edges(edge_ls):
    
    edge_dc = {} 
    
    # Group tuple by verb
    # Result: {verb1: [(sub1, v1, obj1), (sub2, v2, obj2), ...],
    #          verb2: [(sub3, v3, obj3), (sub4, v4, obj4), ...]}
    
    for tup in edge_ls: 
        if tup[1] in edge_dc: 
            edge_dc[tup[1]].append((tup[0], tup[1], tup[2])) 
        else: 
            edge_dc[tup[1]] = [(tup[0], tup[1], tup[2])] 
    
    for edge_labels, tup_ls in tqdm(edge_dc.items()):   # k=edge labels, v = list of tuples
        
        tx = graph.begin()
        
        for el in tup_ls:
            source_node = nodes_matcher.match(name=el[0]).first()
            target_node = nodes_matcher.match(name=el[2]).first()
            if not source_node:
                source_node = Node('Node', name=el[0])
                tx.create(source_node)
            if not target_node:
                try:
                    target_node = Node('Node', name=el[2], node_labels=el[4], url=el[5], word_vec=el[6])
                    tx.create(target_node)
                except:
                    continue
            try:
                rel = Relationship(source_node, edge_labels, target_node)
            except:
                continue
            tx.create(rel)
        tx.commit()
    
    return


def edge_tuple_creation(text):
    
    initial_tup_ls = create_svo_triples(text)
    init_obj_tup_ls = get_obj_properties(initial_tup_ls)
    new_layer_ls = add_layer(init_obj_tup_ls)
    starter_edge_ls = init_obj_tup_ls + new_layer_ls
    edge_ls = subj_equals_obj(starter_edge_ls)
    edges_word_vec_ls = create_word_vectors(edge_ls)
    
    return edges_word_vec_ls


def node_tuple_creation(edges_word_vec_ls):
    
    orig_node_tup_ls = [(edges_word_vec_ls[0][0], '', ['Subject'], '', np.random.uniform(low=-1.0, high=1.0, size=(300,)))]
    obj_node_tup_ls = [(tup[2], tup[3], tup[4], tup[5], tup[6]) for tup in edges_word_vec_ls]
    full_node_tup_ls = orig_node_tup_ls + obj_node_tup_ls
    cleaned_node_tup_ls = check_for_string_labels(full_node_tup_ls)
    #dedup_node_tup_ls = dedup(cleaned_node_tup_ls)
    dedup_node_tup_ls = cleaned_node_tup_ls
    node_tup_ls = convert_vec_to_ls(dedup_node_tup_ls)
    
    return node_tup_ls  

In [66]:
%%time

text = f"""
1. The Afghanistan ‘Transformation Decade’ begins in 2015. With presidential 
elections in 2014, the withdrawal of international troops, and the closure of 
provincial reconstruction teams, Afghanistan continues its journey towards selfreliance. The coming years will be marked by the full sovereignty of Afghanistan
over its political, security and development processes. The international community 
and the United Nations will continue to support Afghanistan, as exemplified in the 
Tokyo Mutual Accountability Framework and the Chicago commitments on 
development and security. As Afghanistan enters its Transformation Decade, UNDP 
should also transition to meet emerging priorities and operational challenges, guided 
by the United Nations Development Assistance Framework and the new strategic 
plan, 2014-2017.
2. Considerable development gains have been made since 2001, despite the 
continued insurgency in parts of the country — a conflict that led to close to 9,000 
civilian casualties in 2013. Presidential elections were held in 2004, 2009 and 2014, 
the writ of the state has expanded across the country, and socio-economic 
development has led to growth in gross domestic product per capita from $186 in 
2002 to $688 in 2012. The Millennium Development Goals Report, 2012, suggests 
that Afghanistan should be able to achieve the Millennium Development Goals for 
education and health by its target year, 2020. Over 30 per cent of central 
government employees are women, and 28 per cent of the seats in the national 
parliament are reserved for women. With regard to proxy indicators used to 
determine multidimensional poverty levels, health and education show considerable 
improvement: infant mortality rates declined by more than 50 per cent between 2003 
and 2012, and net enrolment in primary school rose from 54 per cent in 2003 to 
77 per cent in 2013. 
3. Progress has been uneven, however, across development sectors, between 
income groups, between men and women, and between rural and urban areas, and 
the country still faces conditions of fragility, a reduction of economic growth, and, 
currently, an acute cash crisis. Based on an analysis of development needs and 
guided by national development policies, as reflected in the United Nations 
Development Assistance Framework, the Tokyo Mutual Accountability Framework, 
the national priority programmes and the emerging ‘New Deal’ agenda, as well as 
the lessons learned reflected in the assessment of development results and the 
United Nations common country assessment, the United Nations family and partners 
have agreed to five outcomes to address fragility in its many dimensions and the 
root causes of conflict: (a) equitable economic development; (b) social services; 
(c) social equity and investment in human capital, (d) justice and the rule of law; 
and (e) accountable governance. Responding to the Kabul Conference request for a 
unified United Nations system, the United Nations country team has committed to 
increasing the effectiveness of the system. Furthermore, UNDP is supporting the 
implementation of the New Deal for Engagement in Fragile States principle s with 
the Ministry of Finance. The five New Deal peacebuilding and state building goals, 
leading to conflict transformation, will act as a foundation for progress towards the 
national priority programmes and will guide the Government towards inclusive, 
country-led and country-owned strategies. As part of the New Deal, and in line with 
the Monterrey, Rome, Paris, Accra and Busan aid effectiveness principles, UNDP is 
engaging in better aid management to gradually align development financing with national priorities, donor initiatives, country systems and accountability systems 
such as the Development Assistance Database, and is ensuring harmonized, sectorwide approaches to the allocation of resources.

"""

initial_tup_ls = create_svo_triples(text, print_lists=False)

initial_tup_ls[0:3]

init_obj_tup_ls = get_obj_properties(initial_tup_ls)
new_layer_ls = add_layer(init_obj_tup_ls)
starter_edge_ls = init_obj_tup_ls + new_layer_ls
edge_ls = subj_equals_obj(starter_edge_ls)
#clean_edge_ls = check_for_string_labels(edge_ls)
#clean_edge_ls[0:3]
clean_edge_ls = edge_ls

edge_ls[0:3]

edges_word_vec_ls = create_word_vectors(edge_ls)

orig_node_tup_ls = [(edge_ls[0][0], '', ['Subject'], '', np.random.uniform(low=-1.0, high=1.0, size=(300,)))]
obj_node_tup_ls = [(tup[2], tup[3], tup[4], tup[5], tup[6]) for tup in edges_word_vec_ls]
full_node_tup_ls = orig_node_tup_ls + obj_node_tup_ls
dedup_node_tup_ls = dedup(full_node_tup_ls)

len(full_node_tup_ls), len(dedup_node_tup_ls)
node_tup_ls = convert_vec_to_ls(dedup_node_tup_ls)



CPU times: user 42.8 s, sys: 604 ms, total: 43.4 s
Wall time: 2min 11s


# Populate the graph db



In [67]:
# If you are using a Sandbox instance, you will want to use the following (commented) line.  
# If you are using a Docker container for your DB, use the uncommented line.
# graph = Graph("bolt://some_ip_address:7687", name="neo4j", password="some_password")

graph = Graph("bolt://localhost:7687", name="neo4j", password="")
nodes_matcher = NodeMatcher(graph)

add_nodes(node_tup_ls)
add_edges(edges_word_vec_ls)

Number of nodes in graph:  83


  0%|          | 0/20 [00:00<?, ?it/s]/var/folders/pq/bqmlvyrj2cx73wpm001ps2pr0000gn/T/ipykernel_85561/142667097.py:36: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()
100%|██████████| 20/20 [00:02<00:00,  7.27it/s]


### Updating nodes and edges in graph db

In [ ]:
# %%time
# barack_text = wikipedia.summary('barack obama')
# barack_edges_word_vec_ls = edge_tuple_creation(barack_text)
# barack_node_tup_ls = node_tuple_creation(barack_edges_word_vec_ls)

# michelle_text = wikipedia.summary('michelle obama')
# michelle_edges_word_vec_ls = edge_tuple_creation(michelle_text)
# michelle_node_tup_ls = node_tuple_creation(michelle_edges_word_vec_ls)


# full_node_ls = barack_node_tup_ls + michelle_node_tup_ls
# full_edge_ls = barack_edges_word_vec_ls + michelle_edges_word_vec_ls
# full_dedup_node_tup_ls = dedup(full_node_ls)
# print(len(full_node_ls), len(full_dedup_node_tup_ls))
# add_nodes(full_dedup_node_tup_ls)
# add_edges(full_edge_ls)